In [1]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime, timezone

## TrafficJams - Datum, čas, ulice

In [2]:
trafficDelaysDf = pd.read_csv('C:/Users/verav/Documents/Projekt/Python_projekt/plynulostVSE.csv')

In [3]:
trafficDelaysDf = trafficDelaysDf.drop(columns = ['type'])
trafficDelaysDf.columns = trafficDelaysDf.columns.str.replace('properties.','')
trafficDelaysDf= trafficDelaysDf.rename(columns={'geometry.type':'coordType'})
trafficDelaysDf= trafficDelaysDf.rename(columns={'geometry.coordinates':'coordinates'})

odmazání nepotřebných sloupců

In [4]:
trafficDelaysDf= trafficDelaysDf.drop(['country','endNode','type', 'roadType', 'turnType', 'globalid'], axis = 1)

In [5]:
trafficDelaysDf['speedKMH'] = trafficDelaysDf['speed']*3.6 #převod z m/s na km/h

In [6]:
trafficDelaysDf.head()

,uuid,pubMillis,level,delay,speed,speedKMH,length,street,city,blockingAlertUuid,objectid,coordType,coordinates
0,1363839630,2020/07/13 02:26:04.341+00,5,-1,0,0.0,162,» Centrum / Bohunice,Brno,NaN,1,LineString,"[[16.558271, 49.171838], [16.558371, 49.17184]..."
1,1388730276,2020/07/13 02:26:01.618+00,5,-1,0,0.0,45,NaN,Brno,aff5a9d0-5b84-3878-94a5-4c8c39ccfe4c,2,LineString,"[[16.567145, 49.171995], [16.567112, 49.171732..."
2,1451737922,2020/07/13 02:26:03.702+00,5,-1,0,0.0,52,U Dráhy,Ostopovice,f5987a0c-c5e0-3064-ad5d-9d58b296b1d9,3,LineString,"[[16.544367, 49.16387], [16.544937, 49.163597]]"
3,1388985166,2020/07/13 02:27:45.411+00,5,-1,0,0.0,19,Na Hrázi,Střelice,a7ae280a-44f3-36cc-a6ce-8d5720ac4e37,4,LineString,"[[16.501287, 49.153107], [16.501311, 49.153273]]"
4,1440208034,2020/07/13 02:26:06.591+00,5,-1,0,0.0,44,NaN,Brno,c2b5e9f5-8eef-3e08-9092-ba52f4726a62,5,LineString,"[[16.565195, 49.168989], [16.565247, 49.169382]]"


In [7]:
type(trafficDelaysDf['pubMillis'][0])

str

odmazání milisekund a timezone z pubMillis (jinak problém s převodem na datetime)

In [8]:
dateTimeStrList = []
for i in range(0,len(trafficDelaysDf['pubMillis'])):
    if "." in trafficDelaysDf['pubMillis'][i]:
        dateTimeStrList.append(trafficDelaysDf['pubMillis'][i].split(".")[0])
    elif "+" in trafficDelaysDf['pubMillis'][i]:
        dateTimeStrList.append(trafficDelaysDf['pubMillis'][i].split("+")[0])
    else:
        print('error')

In [ ]:
dateTimeStrList[1000:1020]

převod na dateTime formát

In [9]:
dateTimeList1 = []
for i in range(0,len(dateTimeStrList)):
    dateTimeList1.append(datetime.strptime(dateTimeStrList[i], '%Y/%m/%d %H:%M:%S')) # % = formát ve kterém jsou pův.data

přiřazení info o time zone

In [10]:
dateTimeUTCList = []
for i in range(0,len(dateTimeList1)):
    dateTimeUTCList.append(dateTimeList1[i].replace(tzinfo=timezone.utc))

převod na naši zonu

In [11]:
dateTimeSTCList = []
for i in range(0,len(dateTimeUTCList)):
    dateTimeSTCList.append(dateTimeUTCList[i].astimezone())

In [12]:
dateTimeSTCList[1:5]

[datetime.datetime(2020, 7, 13, 4, 26, 1, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200), 'Střední Evropa (letní čas)')),
 datetime.datetime(2020, 7, 13, 4, 26, 3, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200), 'Střední Evropa (letní čas)')),
 datetime.datetime(2020, 7, 13, 4, 27, 45, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200), 'Střední Evropa (letní čas)')),
 datetime.datetime(2020, 7, 13, 4, 26, 6, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200), 'Střední Evropa (letní čas)'))]

vytvoření nových sloupců (datum,weekdays,seznam hodin, minut, čtvrthodin)

In [13]:
weekDayList = []
weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
for i in range(0,len(dateTimeSTCList)):
    weekDayList.append(weekDays[dateTimeSTCList[i].weekday()]) #.weekday() přiřadí číslo od 0 do 6 (tj.pak index weekDays)

In [14]:
trafficDelaysDf['weekdays'] = weekDayList

In [15]:
hoursList = []
for i in range(0,len(dateTimeSTCList)):
    hoursList.append(dateTimeSTCList[i].hour)

In [16]:
trafficDelaysDf['hours'] = hoursList

In [17]:
minutesList = []
for i in range(0,len(dateTimeSTCList)):
    minutesList.append(dateTimeSTCList[i].minute)

In [18]:
trafficDelaysDf['minutes'] = minutesList

In [19]:
quarters = []
for i in range(0,len(dateTimeSTCList)):
    if trafficDelaysDf['minutes'][i] <= 15:
        quarters.append('00')
    elif trafficDelaysDf['minutes'][i] <= 30:
        quarters.append('15')
    elif trafficDelaysDf['minutes'][i] <= 45:
        quarters.append('30')
    else:
        quarters.append('45')

In [20]:
trafficDelaysDf['quarters'] = quarters

In [21]:
dateTimeList = []# pro přidání do tabulky bez info o time zone
for i in range(0,len(dateTimeSTCList)):
    dateTimeList.append(dateTimeSTCList[i].replace(tzinfo=None))

In [22]:
trafficDelaysDf['datumCas'] = dateTimeList

In [23]:
datum = []
for i in trafficDelaysDf.index:
    date = trafficDelaysDf['datumCas'][i].date()
    datum.append(date)

In [24]:
trafficDelaysDf['datum'] = datum

In [25]:
trafficDelaysDf.head()

,uuid,pubMillis,level,delay,speed,speedKMH,length,street,city,blockingAlertUuid,objectid,coordType,coordinates,weekdays,hours,minutes,quarters,datumCas,datum
0,1363839630,2020/07/13 02:26:04.341+00,5,-1,0,0.0,162,» Centrum / Bohunice,Brno,NaN,1,LineString,"[[16.558271, 49.171838], [16.558371, 49.17184]...",Monday,4,26,15,2020-07-13 04:26:04,2020-07-13
1,1388730276,2020/07/13 02:26:01.618+00,5,-1,0,0.0,45,NaN,Brno,aff5a9d0-5b84-3878-94a5-4c8c39ccfe4c,2,LineString,"[[16.567145, 49.171995], [16.567112, 49.171732...",Monday,4,26,15,2020-07-13 04:26:01,2020-07-13
2,1451737922,2020/07/13 02:26:03.702+00,5,-1,0,0.0,52,U Dráhy,Ostopovice,f5987a0c-c5e0-3064-ad5d-9d58b296b1d9,3,LineString,"[[16.544367, 49.16387], [16.544937, 49.163597]]",Monday,4,26,15,2020-07-13 04:26:03,2020-07-13
3,1388985166,2020/07/13 02:27:45.411+00,5,-1,0,0.0,19,Na Hrázi,Střelice,a7ae280a-44f3-36cc-a6ce-8d5720ac4e37,4,LineString,"[[16.501287, 49.153107], [16.501311, 49.153273]]",Monday,4,27,15,2020-07-13 04:27:45,2020-07-13
4,1440208034,2020/07/13 02:26:06.591+00,5,-1,0,0.0,44,NaN,Brno,c2b5e9f5-8eef-3e08-9092-ba52f4726a62,5,LineString,"[[16.565195, 49.168989], [16.565247, 49.169382]]",Monday,4,26,15,2020-07-13 04:26:06,2020-07-13


In [26]:
trafficDelaysDf.columns

Index(['uuid', 'pubMillis', 'level', 'delay', 'speed', 'speedKMH', 'length',
       'street', 'city', 'blockingAlertUuid', 'objectid', 'coordType',
       'coordinates', 'weekdays', 'hours', 'minutes', 'quarters', 'datumCas',
       'datum'],
      dtype='object')

přehození pořadí sloupců, aby byly souřadnice na konci

In [27]:
trafficDelaysDf = trafficDelaysDf[[col for col in trafficDelaysDf if col not in ['weekdays','hours','minutes','quarters','coordType','coordinates']]+['weekdays','hours','quarters','minutes','coordType','coordinates']] 

přiřazení doplněných názvů ulic z StreetNames.ipynb

In [28]:
streetsDf = pd.read_csv('finalDfstreet.csv', encoding = 'UTF-8')

In [29]:
trafficDelaysDf['street'] = streetsDf['street']

uprava názvů ulic (Žabovřeská) a doplnění města

In [30]:
trafficDelaysDf['street'] = trafficDelaysDf['street'].str.strip()

In [ ]:
trafficDelaysDf[trafficDelaysDf['street'] == "Žabovřesky,okres Brno-město"]

In [33]:
trafficDelaysDf['street'] = trafficDelaysDf['street'].replace("Žabovřesky,okres Brno-město","Žabovřeská")

In [34]:
trafficDelaysDf['city'] = trafficDelaysDf['city'].fillna("Brno")

In [35]:
trafficDelaysDf.head()

,uuid,pubMillis,level,delay,speed,speedKMH,length,street,city,blockingAlertUuid,objectid,datumCas,datum,weekdays,hours,quarters,minutes,coordType,coordinates
0,1363839630,2020/07/13 02:26:04.341+00,5,-1,0,0.0,162,» Centrum / Bohunice,Brno,NaN,1,2020-07-13 04:26:04,2020-07-13,Monday,4,15,26,LineString,"[[16.558271, 49.171838], [16.558371, 49.17184]..."
1,1388730276,2020/07/13 02:26:01.618+00,5,-1,0,0.0,45,Labská,Brno,aff5a9d0-5b84-3878-94a5-4c8c39ccfe4c,2,2020-07-13 04:26:01,2020-07-13,Monday,4,15,26,LineString,"[[16.567145, 49.171995], [16.567112, 49.171732..."
2,1451737922,2020/07/13 02:26:03.702+00,5,-1,0,0.0,52,U Dráhy,Ostopovice,f5987a0c-c5e0-3064-ad5d-9d58b296b1d9,3,2020-07-13 04:26:03,2020-07-13,Monday,4,15,26,LineString,"[[16.544367, 49.16387], [16.544937, 49.163597]]"
3,1388985166,2020/07/13 02:27:45.411+00,5,-1,0,0.0,19,Na Hrázi,Střelice,a7ae280a-44f3-36cc-a6ce-8d5720ac4e37,4,2020-07-13 04:27:45,2020-07-13,Monday,4,15,27,LineString,"[[16.501287, 49.153107], [16.501311, 49.153273]]"
4,1440208034,2020/07/13 02:26:06.591+00,5,-1,0,0.0,44,Okrouhlá,Brno,c2b5e9f5-8eef-3e08-9092-ba52f4726a62,5,2020-07-13 04:26:06,2020-07-13,Monday,4,15,26,LineString,"[[16.565195, 49.168989], [16.565247, 49.169382]]"


In [36]:
trafficDelaysDf.to_csv('trafficDelaysDateStreets.csv', encoding = 'utf-8', index = False)

vytvoření souboru bez souřadnic pro jednodušší import do sql a powerBI

In [37]:
trafficJamsNoCoord = trafficDelaysDf.iloc[:,:-2]

In [38]:
trafficJamsNoCoord.head()

,uuid,pubMillis,level,delay,speed,speedKMH,length,street,city,blockingAlertUuid,objectid,datumCas,datum,weekdays,hours,quarters,minutes
0,1363839630,2020/07/13 02:26:04.341+00,5,-1,0,0.0,162,» Centrum / Bohunice,Brno,NaN,1,2020-07-13 04:26:04,2020-07-13,Monday,4,15,26
1,1388730276,2020/07/13 02:26:01.618+00,5,-1,0,0.0,45,Labská,Brno,aff5a9d0-5b84-3878-94a5-4c8c39ccfe4c,2,2020-07-13 04:26:01,2020-07-13,Monday,4,15,26
2,1451737922,2020/07/13 02:26:03.702+00,5,-1,0,0.0,52,U Dráhy,Ostopovice,f5987a0c-c5e0-3064-ad5d-9d58b296b1d9,3,2020-07-13 04:26:03,2020-07-13,Monday,4,15,26
3,1388985166,2020/07/13 02:27:45.411+00,5,-1,0,0.0,19,Na Hrázi,Střelice,a7ae280a-44f3-36cc-a6ce-8d5720ac4e37,4,2020-07-13 04:27:45,2020-07-13,Monday,4,15,27
4,1440208034,2020/07/13 02:26:06.591+00,5,-1,0,0.0,44,Okrouhlá,Brno,c2b5e9f5-8eef-3e08-9092-ba52f4726a62,5,2020-07-13 04:26:06,2020-07-13,Monday,4,15,26


In [39]:
trafficJamsNoCoord.to_csv('trafficJamsNoCoord1.csv',encoding = "utf-8", index = False)